In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, get_better_notifica
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

timerAll = Timer()
timer = Timer()

In [2]:
timer.start()

notifica = Notifica()
notifica.download_todas_notificacoes()
notifica.save()
# notifica.load()
print(timer.ftime())
notifica.shape()

Requesting notifica where classificacao_final IS NULL
Success code 202
Saving in input\queries\null.csv
Download finish, time elapsed: 0:00:12.805293

normalize notifica
Requesting notifica where classificacao_final = 0
Success code 202
Saving in input\queries\0.csv
Download finish, time elapsed: 0:00:12.597233

normalize notifica
Requesting notifica where classificacao_final = 1
Success code 202
Saving in input\queries\1.csv
Download finish, time elapsed: 0:00:29.096022

normalize notifica
Requesting notifica where classificacao_final = 2
Success code 202
Saving in input\queries\2.csv
Download finish, time elapsed: 0:00:23.262636

normalize notifica
Requesting notifica where classificacao_final = 3
Success code 202
Saving in input\queries\3.csv
Download finish, time elapsed: 0:00:27.995701

normalize notifica
Requesting notifica where classificacao_final = 5
Success code 202
Saving in input\queries\5.csv
Download finish, time elapsed: 0:00:26.327610

normalize notifica
00:23:02.810


(2238828, 1028707, 10529, 1197502, 2090)

In [3]:
timer.start()
casos_confirmados = CasosConfirmados()
casos_confirmados.read()
casos_confirmados.save()
# casos_confirmados.load()
print(timer.ftime())
casos_confirmados.shape()

00:02:31.338


(569911, 10388, 420248, 139275)

In [4]:
casosn = notifica.get_casos()
casosn['duplicado'] = 0
casosc = casos_confirmados.get_casos()
casosn.groupby('classificacao_final')[['id']].count()

,id
classificacao_final,
CASO CONFIRMADO,542804
CASO DESCARTADO,764704
CASO SUSPEITO,923667
IGNORADO,641
SINDROME GRIPAL NAO ESPECIFICADA,7012


In [5]:
casos_duplicados = casosn.loc[(casosn['cpf'].notnull()) & (casosn.duplicated('cpf',keep=False))].sort_values('cpf')
print(f"Total de pacientes com mais de uma ocorrencia: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('cpf'))} pacientes pelo CPF que estavam com mais de uma ocorrencia")

all_casos_duplicados = set()
all_duplicados_mantidos = set()
manter = []

for hash, group in casos_duplicados.groupby('cpf'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_cpf.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia: 499965
201453 pacientes pelo CPF que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,14352
CASO DESCARTADO,123404
CASO SUSPEITO,159306
IGNORADO,48
SINDROME GRIPAL NAO ESPECIFICADA,1402


In [6]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [7]:
casos_duplicados = casosn.loc[(casosn['nome_mae'].notnull()) & (casosn.duplicated('hash_mae',keep=False))].sort_values('nome_mae')

print(f"Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('nome_mae'))} pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('nome_mae'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_mae.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: 538656
197934 pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,20414
CASO DESCARTADO,143090
CASO SUSPEITO,175624
IGNORADO,39
SINDROME GRIPAL NAO ESPECIFICADA,1555


In [8]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [9]:
casos_duplicados = casosn.loc[(casosn['hash_nasc'].notnull()) & (casosn.duplicated('hash_nasc',keep=False))].sort_values('data_nascimento')

print(f"Total de pacientes com mais de uma ocorrencia pelo data_nascimento: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_nasc'))} pacientes pelo data_nascimento que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_nasc'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_data_nascimento.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo data_nascimento: 575415
233749 pacientes pelo data_nascimento que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,17357
CASO DESCARTADO,139347
CASO SUSPEITO,183333
IGNORADO,50
SINDROME GRIPAL NAO ESPECIFICADA,1579


In [10]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [11]:
casos_duplicados = casosn.loc[(casosn['hash_resid'].notnull()) & (casosn.duplicated('hash_resid',keep=False))].sort_values('paciente')

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_resid: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_resid'))} pacientes pelo hash_resid que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_resid'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_resid.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_resid: 469172
191823 pacientes pelo hash_resid que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,14919
CASO DESCARTADO,112920
CASO SUSPEITO,148264
IGNORADO,34
SINDROME GRIPAL NAO ESPECIFICADA,1212


In [12]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [13]:
casos_duplicados = casosn.loc[(casosn['hash_atend'].notnull()) & (casosn.duplicated('hash_atend',keep=False))].sort_values('paciente')
# casos_duplicados_nome_mae.loc[casos_duplicados_nome_mae['nome_mae']=='0']

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_atend: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_atend'))} pacientes pelo hash_atend que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_atend'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_atend.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_atend: 402722
163750 pacientes pelo hash_atend que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,11637
CASO DESCARTADO,100255
CASO SUSPEITO,125934
IGNORADO,18
SINDROME GRIPAL NAO ESPECIFICADA,1128


In [14]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [15]:
casos_duplicados = casosn.loc[(casosn['hash_diag'].notnull()) & (casosn.duplicated('hash_diag',keep=False))].sort_values('paciente')

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_diag: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_diag'))} pacientes pelo hash_diag que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_diag'):
    idx = get_better_notifica(group)
    manter.append(idx)


casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_hash_diag.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()

Total de pacientes com mais de uma ocorrencia pelo hash_diag: 51505
24698 pacientes pelo hash_diag que estavam com mais de uma ocorrencia


,id
classificacao_final,
CASO CONFIRMADO,4743
CASO DESCARTADO,10129
CASO SUSPEITO,11881
IGNORADO,11
SINDROME GRIPAL NAO ESPECIFICADA,43


In [16]:
casosn.loc[casos_duplicados, 'duplicado'] = 1 
# casosn = casosn.drop(index=casos_duplicados)

In [17]:
notifica.save(casosn)

In [18]:
print(f"Das {notifica.shape()[0]} notificações baixadas, {notifica.shape()[0] - casosn.shape[0]} ({len(all_casos_duplicados)}) foram excluidas por serem do mesmo paciente, sendo mantida a que melhor pontuou nos critérios. Sendo assim, o novo total de notificações existente é de {casosn.shape[0]}")
allcasosn = notifica.get_casos()
allcasosn.loc[all_casos_duplicados,'id'].to_csv(join(output,'casos_duplicados_removidos.csv'), index=False)
allcasosn.loc[all_duplicados_mantidos,'id'].to_csv(join(output,'casos_duplicados_mantidos.csv'), index=False)

Das 2238828 notificações baixadas, 0 (389195) foram excluidas por serem do mesmo paciente, sendo mantida a que melhor pontuou nos critérios. Sendo assim, o novo total de notificações existente é de 2238828


In [19]:
idx_casos_confirmados = casosc[casosc['hash_resid'].isin(casosn['hash_resid'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casosn['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_diag'].isin(casosn['hash_diag'])].index.tolist()

idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_notifica = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')
casos_confirmados_nao_notifica.to_excel(join(output,'casos_confirmados_nao_notifica.xlsx'),index=False)
casos_confirmados_nao_notifica.shape

(27977, 23)

In [20]:
idx_casos_notifica = casosn[casosn['hash_resid'].isin(casosc['hash_resid'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_casos_notifica += casosn[casosn['hash_diag'].isin(casosc['hash_diag'])].index.tolist()

idx_casos_notifica = set(idx_casos_notifica)
casos_confirmados_notifica = casosn.loc[ idx_casos_notifica ].sort_values('id')
casos_confirmados_notifica.to_excel(join(output,'casos_confirmados_notifica.xlsx'),index=False)
casos_confirmados_notifica.shape

(629705, 54)

In [21]:
notifica_nao_casos_confirmados = casosn.loc[set(casosn.index.tolist()) - set(idx_casos_notifica)]

In [22]:
notifica_nao_casos_confirmados = notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['cod_classificacao_final'] == 2]
notifica_nao_casos_confirmados = notifica_nao_casos_confirmados.loc[notifica_nao_casos_confirmados['cod_status_notificacao'].isin([1,2])]

notifica_nao_casos_confirmados['month'] = notifica_nao_casos_confirmados.apply(lambda x: x['data_notificacao'].month, axis=1)
nnccm = notifica_nao_casos_confirmados.groupby(by='month')[['id']].count()
nnccm.index = [ meses[int(x)-1] for x in nnccm.index] 
nnccm

,id
jan,535
fev,384
mar,41
abril,106
mai,140
jun,454
jul,529
ago,378
set,204
out,148


In [23]:
notifica_nao_casos_confirmados.to_excel(join(output,'notifica_nao_casos_confirmados.xlsx'), index=False)
notifica_nao_casos_confirmados.shape

(3548, 55)

In [25]:
casosn.loc[casosn['hash_diag']=='WESLEIZOLET05022021']

,id,paciente,data_nascimento,nome_mae,cpf,cod_tipo_paciente,tipo_paciente,idade,sexo,cod_raca_cor,raca_cor,cod_etnia,etnia,uf_residencia,ibge_residencia,cod_classificacao_final,classificacao_final,cod_criterio_classificacao,criterio_classificacao,cod_evolucao,evolucao,data_1o_sintomas,data_cura_obito,co_seq_exame,cod_metodo,metodo,cod_exame,exame,cod_resultado,resultado,data_coleta,data_recebimento,data_liberacao,cod_status_notificacao,status_notificacao,excluir_ficha,cod_origem,origem,uf_unidade_notifica,ibge_unidade_notifica,data_notificacao,updated_at,data_diagnostico,mun_resid,uf_resid,rs,mun_atend,uf_atend,hash_resid,hash_atend,hash_mae,hash_nasc,hash_diag,duplicado
1457664,2254867,WESLEI ZOLET,1996-12-14,MARIA DE LOURDES DA SILVA,107.557.909-01,3,CPF INFORMADO,24,M,99,IGNORADO,0,,41,412770,2,CASO CONFIRMADO,1,LABORATORIAL,3,NAO SE APLICA,2021-01-28,NaT,5538724,1,RT-PCR,27,COVID-19 BIOLOGIA MOLECULAR,9,CORONAVIRUS (SARS-COV2),2021-02-02,2021-02-04,2021-02-05,1,ABERTA,2,1,NOTIFICA COVID-19,41,412770,2021-02-02,2021-02-06,2021-02-05,TOLEDO,PR,20,TOLEDO,PR,WESLEIZOLET24TOLEDO,WESLEIZOLET24TOLEDO,WESLEIZOLETMARIADELOURDESDASILVA,WESLEIZOLET14121996,WESLEIZOLET05022021,1


In [26]:
casosn.loc[casosn['hash_resid']=='WESLEIZOLET24TOLEDO']

,id,paciente,data_nascimento,nome_mae,cpf,cod_tipo_paciente,tipo_paciente,idade,sexo,cod_raca_cor,raca_cor,cod_etnia,etnia,uf_residencia,ibge_residencia,cod_classificacao_final,classificacao_final,cod_criterio_classificacao,criterio_classificacao,cod_evolucao,evolucao,data_1o_sintomas,data_cura_obito,co_seq_exame,cod_metodo,metodo,cod_exame,exame,cod_resultado,resultado,data_coleta,data_recebimento,data_liberacao,cod_status_notificacao,status_notificacao,excluir_ficha,cod_origem,origem,uf_unidade_notifica,ibge_unidade_notifica,data_notificacao,updated_at,data_diagnostico,mun_resid,uf_resid,rs,mun_atend,uf_atend,hash_resid,hash_atend,hash_mae,hash_nasc,hash_diag,duplicado
1457664,2254867,WESLEI ZOLET,1996-12-14,MARIA DE LOURDES DA SILVA,107.557.909-01,3,CPF INFORMADO,24,M,99,IGNORADO,0,,41,412770,2,CASO CONFIRMADO,1,LABORATORIAL,3,NAO SE APLICA,2021-01-28,NaT,5538724,1,RT-PCR,27,COVID-19 BIOLOGIA MOLECULAR,9,CORONAVIRUS (SARS-COV2),2021-02-02,2021-02-04,2021-02-05,1,ABERTA,2,1,NOTIFICA COVID-19,41,412770,2021-02-02,2021-02-06,2021-02-05,TOLEDO,PR,20,TOLEDO,PR,WESLEIZOLET24TOLEDO,WESLEIZOLET24TOLEDO,WESLEIZOLETMARIADELOURDESDASILVA,WESLEIZOLET14121996,WESLEIZOLET05022021,1


In [27]:
casosn.loc[casosn['hash_atend']=='WESLEIZOLET24TOLEDO']

,id,paciente,data_nascimento,nome_mae,cpf,cod_tipo_paciente,tipo_paciente,idade,sexo,cod_raca_cor,raca_cor,cod_etnia,etnia,uf_residencia,ibge_residencia,cod_classificacao_final,classificacao_final,cod_criterio_classificacao,criterio_classificacao,cod_evolucao,evolucao,data_1o_sintomas,data_cura_obito,co_seq_exame,cod_metodo,metodo,cod_exame,exame,cod_resultado,resultado,data_coleta,data_recebimento,data_liberacao,cod_status_notificacao,status_notificacao,excluir_ficha,cod_origem,origem,uf_unidade_notifica,ibge_unidade_notifica,data_notificacao,updated_at,data_diagnostico,mun_resid,uf_resid,rs,mun_atend,uf_atend,hash_resid,hash_atend,hash_mae,hash_nasc,hash_diag,duplicado
1457664,2254867,WESLEI ZOLET,1996-12-14,MARIA DE LOURDES DA SILVA,107.557.909-01,3,CPF INFORMADO,24,M,99,IGNORADO,0,,41,412770,2,CASO CONFIRMADO,1,LABORATORIAL,3,NAO SE APLICA,2021-01-28,NaT,5538724,1,RT-PCR,27,COVID-19 BIOLOGIA MOLECULAR,9,CORONAVIRUS (SARS-COV2),2021-02-02,2021-02-04,2021-02-05,1,ABERTA,2,1,NOTIFICA COVID-19,41,412770,2021-02-02,2021-02-06,2021-02-05,TOLEDO,PR,20,TOLEDO,PR,WESLEIZOLET24TOLEDO,WESLEIZOLET24TOLEDO,WESLEIZOLETMARIADELOURDESDASILVA,WESLEIZOLET14121996,WESLEIZOLET05022021,1


In [30]:
casosn.loc[casosn['hash_mae']=='WESLEIZOLETMARIADELOURDESDASILVA']

,id,paciente,data_nascimento,nome_mae,cpf,cod_tipo_paciente,tipo_paciente,idade,sexo,cod_raca_cor,raca_cor,cod_etnia,etnia,uf_residencia,ibge_residencia,cod_classificacao_final,classificacao_final,cod_criterio_classificacao,criterio_classificacao,cod_evolucao,evolucao,data_1o_sintomas,data_cura_obito,co_seq_exame,cod_metodo,metodo,cod_exame,exame,cod_resultado,resultado,data_coleta,data_recebimento,data_liberacao,cod_status_notificacao,status_notificacao,excluir_ficha,cod_origem,origem,uf_unidade_notifica,ibge_unidade_notifica,data_notificacao,updated_at,data_diagnostico,mun_resid,uf_resid,rs,mun_atend,uf_atend,hash_resid,hash_atend,hash_mae,hash_nasc,hash_diag,duplicado
1457664,2254867,WESLEI ZOLET,1996-12-14,MARIA DE LOURDES DA SILVA,107.557.909-01,3,CPF INFORMADO,24,M,99,IGNORADO,0,,41,412770,2,CASO CONFIRMADO,1,LABORATORIAL,3,NAO SE APLICA,2021-01-28,NaT,5538724,1,RT-PCR,27,COVID-19 BIOLOGIA MOLECULAR,9,CORONAVIRUS (SARS-COV2),2021-02-02,2021-02-04,2021-02-05,1,ABERTA,2,1,NOTIFICA COVID-19,41,412770,2021-02-02,2021-02-06,2021-02-05,TOLEDO,PR,20,TOLEDO,PR,WESLEIZOLET24TOLEDO,WESLEIZOLET24TOLEDO,WESLEIZOLETMARIADELOURDESDASILVA,WESLEIZOLET14121996,WESLEIZOLET05022021,1
1563111,402557,WESLEI ZOLET,1996-12-14,MARIA DE LOURDES DA SILVA,107.557.909-01,3,CPF INFORMADO,23,M,1,BRANCA,0,,41,412770,3,CASO DESCARTADO,1,LABORATORIAL,1,CURA,2020-07-10,2020-07-22,0,1,RT-PCR,27,COVID-19 BIOLOGIA MOLECULAR,11,NEGATIVO,2020-07-15,2020-07-18,2020-07-18,3,INATIVA,2,1,NOTIFICA COVID-19,41,412770,2020-07-15,2020-07-22,2020-07-18,TOLEDO,PR,20,TOLEDO,PR,WESLEIZOLET23TOLEDO,WESLEIZOLET23TOLEDO,WESLEIZOLETMARIADELOURDESDASILVA,WESLEIZOLET14121996,WESLEIZOLET18072020,1


In [28]:
casosn.loc[casosn['hash_nasc']=='WESLEIZOLET14121996']

,id,paciente,data_nascimento,nome_mae,cpf,cod_tipo_paciente,tipo_paciente,idade,sexo,cod_raca_cor,raca_cor,cod_etnia,etnia,uf_residencia,ibge_residencia,cod_classificacao_final,classificacao_final,cod_criterio_classificacao,criterio_classificacao,cod_evolucao,evolucao,data_1o_sintomas,data_cura_obito,co_seq_exame,cod_metodo,metodo,cod_exame,exame,cod_resultado,resultado,data_coleta,data_recebimento,data_liberacao,cod_status_notificacao,status_notificacao,excluir_ficha,cod_origem,origem,uf_unidade_notifica,ibge_unidade_notifica,data_notificacao,updated_at,data_diagnostico,mun_resid,uf_resid,rs,mun_atend,uf_atend,hash_resid,hash_atend,hash_mae,hash_nasc,hash_diag,duplicado
1457664,2254867,WESLEI ZOLET,1996-12-14,MARIA DE LOURDES DA SILVA,107.557.909-01,3,CPF INFORMADO,24,M,99,IGNORADO,0,,41,412770,2,CASO CONFIRMADO,1,LABORATORIAL,3,NAO SE APLICA,2021-01-28,NaT,5538724,1,RT-PCR,27,COVID-19 BIOLOGIA MOLECULAR,9,CORONAVIRUS (SARS-COV2),2021-02-02,2021-02-04,2021-02-05,1,ABERTA,2,1,NOTIFICA COVID-19,41,412770,2021-02-02,2021-02-06,2021-02-05,TOLEDO,PR,20,TOLEDO,PR,WESLEIZOLET24TOLEDO,WESLEIZOLET24TOLEDO,WESLEIZOLETMARIADELOURDESDASILVA,WESLEIZOLET14121996,WESLEIZOLET05022021,1
1563111,402557,WESLEI ZOLET,1996-12-14,MARIA DE LOURDES DA SILVA,107.557.909-01,3,CPF INFORMADO,23,M,1,BRANCA,0,,41,412770,3,CASO DESCARTADO,1,LABORATORIAL,1,CURA,2020-07-10,2020-07-22,0,1,RT-PCR,27,COVID-19 BIOLOGIA MOLECULAR,11,NEGATIVO,2020-07-15,2020-07-18,2020-07-18,3,INATIVA,2,1,NOTIFICA COVID-19,41,412770,2020-07-15,2020-07-22,2020-07-18,TOLEDO,PR,20,TOLEDO,PR,WESLEIZOLET23TOLEDO,WESLEIZOLET23TOLEDO,WESLEIZOLETMARIADELOURDESDASILVA,WESLEIZOLET14121996,WESLEIZOLET18072020,1
